In [1]:
import math
import os
from datetime import datetime
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
def natural_sort_key(s):
    """
    Create a sort key for natural string sorting (e.g., 'item2' before 'item10').
    
    Args:
        s (str): The string to create a key for.
        
    Returns:
        list: A list of strings and integers for sorting.
    """
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def create_plot_grid_pdf(source_folder, file_prefix, n_cols, filename="A4_plots_grid.pdf", sort_order=None, dpi=300, main_title=None, subtitle=None):
    """
    Generates an A4 PDF file with plots from image files arranged in a grid.

    It searches a folder for  files, sorts them, and arranges them in a
    grid. If the last row is not full, the plots are centered.

    Args:
        source_folder (str): The path to the folder containing the plot images.
        file_prefix (str): The prefix of the  files to include.
        n_cols (int): The desired number of columns in the grid.
        filename (str, optional): The name of the output PDF file.
        sort_order (list, optional): A list of the uncommon parts of filenames
            to specify a custom order. E.g., ['02', '01']. If None,
            natural sorting is used.
        dpi (int, optional): The resolution in dots per inch for the output PDF.
        main_title (str, optional): A main title for the entire page.
        subtitle (str, optional): A subtitle to appear below the main title.
    """
    # --- 1. Find and Sort Image Files ---
    search_path = os.path.join(source_folder, f"{file_prefix}*")
    all_files = glob.glob(search_path)

    if not all_files:
        print(f"Error: No '' files found in '{source_folder}' with prefix '{file_prefix}'.")
        return

    if sort_order:
        # Use the user-provided custom sort order
        file_map = {os.path.basename(f).replace(file_prefix, ''): f for f in all_files}
        plot_files = []
        for uncommon_part in sort_order:
            # Add the  extension if the user didn't provide it
            if not uncommon_part.endswith(''):
                uncommon_part += ''
            if uncommon_part in file_map:
                plot_files.append(file_map[uncommon_part])
            else:
                print(f"Warning: File part '{uncommon_part}' from sort_order not found.")
    else:
        # Default to natural sorting
        plot_files = sorted(all_files, key=lambda f: natural_sort_key(os.path.basename(f)))

    total_plots = len(plot_files)
    if total_plots == 0:
        print("Error: No valid plot files to process after sorting.")
        return
    if n_cols <= 0:
        print("Error: Number of columns must be a positive integer.")
        return

    # --- 2. Calculate Layout and Dimensions ---
    A4_WIDTH, A4_HEIGHT = 8.27, 11.69
    # LEFT_MARGIN, RIGHT_MARGIN, BOTTOM_MARGIN = 0.75, 0.5, 0.5 # 0, 0, 0
    LEFT_MARGIN, RIGHT_MARGIN, BOTTOM_MARGIN = 0, 0, 0
    # H_SPACE, V_SPACE = 0.4, 0.5 # 0.05, 0.1
    H_SPACE, V_SPACE = 0.05, 0.1

    # Adjust top margin to make space for the title and subtitle
    # TOP_MARGIN = 0.75 # 0
    TOP_MARGIN = 0
    if main_title:
        TOP_MARGIN += 0.75  # Add space for title, subtitle, and line

    n_rows = math.ceil(total_plots / n_cols)
    drawable_width = A4_WIDTH - LEFT_MARGIN - RIGHT_MARGIN
    drawable_height = A4_HEIGHT - TOP_MARGIN - BOTTOM_MARGIN
    plot_width = (drawable_width - (n_cols - 1) * H_SPACE) / n_cols
    plot_height = (drawable_height - (n_rows - 1) * V_SPACE) / n_rows

    if plot_width <= 0 or plot_height <= 0:
        print("Error: Too many plots or columns for the page size and margins.")
        return

    print(f"Generating '{filename}' with {total_plots} plots in a {n_rows}x{n_cols} grid (DPI: {dpi}).")

    # --- 3. Create PDF and Figure ---
    with PdfPages(filename) as pdf:
        fig = plt.figure(figsize=(A4_WIDTH, A4_HEIGHT))

        # --- Add Page Title and Subtitle ---
        if main_title:
            title_y_pos = (A4_HEIGHT - 0.5) / A4_HEIGHT
            fig.text(0.5, title_y_pos, main_title, ha='center', va='center', fontsize=20, weight='bold')
            
            if subtitle:
                subtitle_y_pos = (A4_HEIGHT - 0.8) / A4_HEIGHT
                fig.text(0.5, subtitle_y_pos, subtitle, ha='center', va='center', fontsize=14)
            
            # Add a horizontal line below the titles
            line_y = (A4_HEIGHT - 1.1) / A4_HEIGHT
            line = plt.Line2D([LEFT_MARGIN/A4_WIDTH, (A4_WIDTH-RIGHT_MARGIN)/A4_WIDTH], 
                              [line_y, line_y], color='black', linewidth=1)
            fig.add_artist(line)

        plot_counter = 0

        # --- 4. Iterate and Place Plots ---
        for r in range(n_rows):
            plots_in_this_row = min(n_cols, total_plots - plot_counter)
            row_content_width = (plots_in_this_row * plot_width +
                                 max(0, plots_in_this_row - 1) * H_SPACE)
            start_x_abs = LEFT_MARGIN + (drawable_width - row_content_width) / 2.0
            y_pos_abs = (A4_HEIGHT - TOP_MARGIN - plot_height -
                         r * (plot_height + V_SPACE))

            for c in range(plots_in_this_row):
                if plot_counter >= total_plots:
                    break

                x_pos_abs = start_x_abs + c * (plot_width + H_SPACE)
                rect = [
                    x_pos_abs / A4_WIDTH,
                    y_pos_abs / A4_HEIGHT,
                    plot_width / A4_WIDTH,
                    plot_height / A4_HEIGHT,
                ]

                # --- 5. Add Image to the Axes ---
                ax = fig.add_axes(rect)
                image_path = plot_files[plot_counter]
                try:
                    img = plt.imread(image_path)
                    ax.imshow(img)
                    # ax.set_title(os.path.basename(image_path), fontsize=8, y=1.02)
                except Exception as e:
                    ax.text(0.5, 0.5, f"Error loading:\n{os.path.basename(image_path)}", 
                            ha='center', va='center', color='red')
                    print(f"Warning: Could not load image {image_path}. Error: {e}")

                ax.axis('off')
                plot_counter += 1

        pdf.savefig(fig, dpi=dpi)
        print("PDF generation complete.")
        plt.close(fig)

In [ ]:
# Distribution Plot for Each Trigger
if __name__ == "__main__":
    suffix = "analyse"

    signal_triggers = [
        "HLT_AK8PFHT800_TrimMass50",
        "HLT_AK8PFJet400_TrimMass30",
        "HLT_AK8PFJet500",
        "HLT_PFJet500",
        "HLT_PFHT1050",
        "HLT_PFHT500_PFMET100_PFMHT100_IDTight",
        "HLT_PFHT700_PFMET85_PFMHT85_IDTight",
        "HLT_PFHT800_PFMET75_PFMHT75_IDTight",
        "HLT_DoubleMediumChargedIsoPFTauHPS35_Trk1_eta2p1_Reg",
        "HLT_MediumChargedIsoPFTau180HighPtRelaxedIso_Trk50_eta2p1"
    ]

    date = datetime.now().strftime('%d-%m-%Y')

    for trigger in signal_triggers:
        main_title = f"New Samples - {trigger}"
        DUMMY_FOLDER = f"result/{date}/{date}-{suffix}_{trigger}"
        FILE_PREFIX = "Distribution_MC_"

        # Custom sort order for distribution data
        print(f"\n--- Running with custom sorted order and a title for {trigger} ---")

        # For Distribution plots
        custom_order = ['HighestPt',
                        'HT',
                        'SecondHighestPt',
                        'MET_pt',
                        'mHH',
                        'mHHwithMET',
                        'HighestMass',
                        'SecondHighestMass',
                        'FatHT',
                        'MET_FatJet',
                        'HighestEta',
                        'SecondHighestEta',
                        'DeltaEta',
                        'DeltaPhi']
        
        custom_order = [t + ".png" for t in custom_order]

        dirs = f"report/{date}/{suffix}"
        if not os.path.exists(dirs):
            os.makedirs(dirs, exist_ok=True)

        filename = f"{dirs}/{trigger}-distribution-simulation.pdf"
        print(f"Creating plot grid for distribution with filename: {filename}")

        create_plot_grid_pdf(
            source_folder=DUMMY_FOLDER,
            file_prefix=FILE_PREFIX,
            n_cols=3,
            filename=filename,
            main_title=main_title,
            subtitle="Feature Distribution",
            sort_order=custom_order
        )

In [ ]:
# Distribution Plot Composing
if __name__ == '__main__':
    version = "v2"
    
    if version == "v1":
        suffix = "briar"
        main_title = "Old Samples"
    elif version == "v2":
        suffix = "azura"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v3":
        suffix = "ashe"
        main_title = "New Samples - No Tau Signal Triggers"
    elif version == "v4":
        suffix = "cypress"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v5":
        suffix = "testing"
        main_title = "New Samples - Tau Signal Triggers"

        
    # Create a folder and some sample  files
    date = datetime.now().strftime('%d-%m-%Y')
    # DUMMY_FOLDER = f"result/{date}-{suffix}"
    DUMMY_FOLDER = f"result/24-07-2025/24-07-2025-{suffix}"
    # FILE_PREFIX = "Distribution_MC_"
    FILE_PREFIX = "DATA_"

    # Example with custom sort order and no title
    print("\n--- Running with custom sorted order and a title ---")

    # For Distribution plots
    custom_order = ['HighestPt', 
                    'HT',
                    'SecondHighestPt', 
                    'MET_pt', 
                    'mHH', 
                    'mHHwithMET', 
                    'HighestMass', 
                    'SecondHighestMass', 
                    'FatHT', 
                    'MET_FatJet', 
                    'HighestEta', 
                    'SecondHighestEta', 
                    'DeltaEta', 
                    'DeltaPhi']
    
    # custom_order = [t + ".png" for t in custom_order]
    custom_order = [t + "_Run2_both.png" for t in custom_order]
        
    dirs = f"report/{date}/{suffix}"
    if not os.path.exists(dirs):
        os.makedirs(dirs, exist_ok=True)

    filename = f"{dirs}/distribution-data-{suffix}.pdf"
    print(f"Creating plot grid for distribution with filename: {filename}")
    create_plot_grid_pdf(
        source_folder=DUMMY_FOLDER,
        file_prefix=FILE_PREFIX,
        n_cols=3,
        filename=filename,
        # main_title=main_title,
        # subtitle="Feature Distribution",
        sort_order=custom_order
    )

In [ ]:
# Separate Sample Distribution Plots
if __name__ == '__main__':
    # samples = ["QCD", "ggF", "VBF", "DATA"]
    samples = ["DATA"]
    version = "v5"
    run_name = "Run3_both"
    
    if version == "v1":
        suffix = "briar"
        main_title = "Old Samples"
    elif version == "v2":
        suffix = "azura"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v3":
        suffix = "ashe"
        main_title = "New Samples - No Tau Signal Triggers"
    elif version == "v4":
        suffix = "cypress"
        main_title = "New Samples - Tau Signal Triggers"
    else:
        suffix = "testing"
        main_title = "New Samples - Tau Signal Triggers"

    # Create a folder and some sample  files
    DUMMY_FOLDER = f"result/{datetime.now().strftime('%d-%m-%Y')}-{suffix}"

    for sample in samples:
        FILE_PREFIX = f"{sample}_"

        # Example with custom sort order and no title
        print("\n--- Running with custom sorted order and a title ---")
    
        # For STD plots
        files = ['HighestPt_', 
                'HT_', 
                'SecondHighestPt_', 
                'MET_pt_', 
                'mHH_', 
                'mHHwithMET_', 
                'HighestMass_', 
                'SecondHighestMass_', 
                'FatHT_', 
                'MET_FatJet_', 
                'HighestEta_', 
                'SecondHighestEta_', 
                'DeltaEta_', 
                'DeltaPhi_']

        reference_trigger = 'HLT_AK8PFJet260'
        type = '.png'
    
        custom_order = [t + run_name + type for t in files]

        dirs = f"report/{datetime.now().strftime('%d-%m-%Y')}/{suffix}"
        if not os.path.exists(dirs):
            os.makedirs(dirs, exist_ok=True)
            # filename.mkdirs(parents=True, exist_ok=True)

        filename = f"{dirs}/{sample}-distribution-{suffix}.pdf"
        print(f"Creating plot grid for {sample} with filename: {filename}")
        create_plot_grid_pdf(
            source_folder=DUMMY_FOLDER,
            file_prefix=FILE_PREFIX,
            n_cols=3,
            filename=filename,
            main_title=main_title,
            subtitle=f"{sample} Distribution Analysis",
            sort_order=custom_order
        )

In [ ]:
# Measured and Real Efficiency Plots
if __name__ == '__main__':
    samples = ["QCD", "ggF", "VBF"]
    version = "v6"
    run_name = "Run2_"
    
    if version == "v1":
        suffix = "briar"
        main_title = "Old Samples"
    elif version == "v2":
        suffix = "azura"
        main_title = "New Samples - Tau Triggers"
    elif version == "v3":
        suffix = "ashe"
        main_title = "New Samples - No Tau Signal Triggers"
    elif version == "v4":
        suffix = "cypress"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v5":
        suffix = "testing"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v6":
        suffix = "newQCD"
        main_title = "New Samples - No Tau Signal Triggers"
        # main_title = "Old Samples"

    # Create a folder and some sample  files
    DUMMY_FOLDER = f"result/{datetime.now().strftime('%d-%m-%Y')}-{suffix}"

    for sample in samples:
        FILE_PREFIX = f"meas_{sample}_"

        # Example with custom sort order and no title
        print("\n--- Running with custom sorted order and a title ---")
    
        # For STD plots
        files = ['HighestPt_', 
                'HT_', 
                'SecondHighestPt_', 
                'MET_pt_', 
                'mHH_', 
                'mHHwithMET_', 
                'HighestMass_', 
                'SecondHighestMass_', 
                'FatHT_', 
                'MET_FatJet_', 
                'HighestEta_', 
                'SecondHighestEta_', 
                'DeltaEta_', 
                'DeltaPhi_']

        reference_trigger = 'HLT_AK8PFJet260'
        type = '.png'
    
        custom_order = [t + run_name + type for t in files]

        dirs = f"report/{datetime.now().strftime('%d-%m-%Y')}/{suffix}"
        if not os.path.exists(dirs):
            os.makedirs(dirs, exist_ok=True)
            # filename.mkdirs(parents=True, exist_ok=True)

        filename = f"{dirs}/meas_{sample}-distribution-{suffix}.pdf"
        print(f"Creating plot grid for {sample} with filename: {filename}")
        create_plot_grid_pdf(
            source_folder=DUMMY_FOLDER,
            file_prefix=FILE_PREFIX,
            n_cols=3,
            filename=filename,
            main_title=main_title,
            subtitle=f"{sample} Real Efficiency",
            sort_order=custom_order
        )

    for sample in samples:
            FILE_PREFIX = f"real_{sample}_"

            # Example with custom sort order and no title
            print("\n--- Running with custom sorted order and a title ---")
        
            # For STD plots
            files = ['HighestPt_', 
                    'HT_', 
                    'SecondHighestPt_', 
                    'MET_pt_', 
                    'mHH_', 
                    'mHHwithMET_', 
                    'HighestMass_', 
                    'SecondHighestMass_', 
                    'FatHT_', 
                    'MET_FatJet_', 
                    'HighestEta_', 
                    'SecondHighestEta_', 
                    'DeltaEta_', 
                    'DeltaPhi_']

            reference_trigger = 'HLT_AK8PFJet260'
            type = '.png'
        
            custom_order = [t + run_name + type for t in files]

            dirs = f"report/{datetime.now().strftime('%d-%m-%Y')}/{suffix}"
            if not os.path.exists(dirs):
                os.makedirs(dirs, exist_ok=True)
                # filename.mkdirs(parents=True, exist_ok=True)

            filename = f"{dirs}/real_{sample}-distribution-{suffix}.pdf"
            print(f"Creating plot grid for {sample} with filename: {filename}")
            create_plot_grid_pdf(
                source_folder=DUMMY_FOLDER,
                file_prefix=FILE_PREFIX,
                n_cols=3,
                filename=filename,
                main_title=main_title,
                subtitle=f"{sample} Measure Efficiency",
                sort_order=custom_order
            )

In [ ]:
# Efficiency Plots
if __name__ == '__main__':
    samples = ["QCD", "ggF", "VBF"]
    version = "v7"
    
    if version == "v1":
        suffix = "briar"
        main_title = "Old Samples"
    elif version == "v2":
        suffix = "azura"
        main_title = "New Samples - Tau Triggers"
    elif version == "v3":
        suffix = "ashe"
        main_title = "New Samples - No Tau Signal Triggers"
    elif version == "v4":
        suffix = "cypress"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v5":
        suffix = "testing"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v6":
        suffix = "newQCD"
        main_title = "New Samples - No Tau Signal Triggers"
    elif version == "v7":
        suffix = "oldQCD"
        main_title = "Old Samples"

    # Create a folder and some sample  files
    DUMMY_FOLDER = "result/03-09-2025-" + suffix

    for sample in samples:
        FILE_PREFIX = f"{sample}_TEfficiency_"

        # Example with custom sort order and no title
        print("\n--- Running with custom sorted order and a title ---")
    
        # For STD plots
        files = ['HighestPt_', 
                'HT_', 
                'SecondHighestPt_', 
                'MET_pt_', 
                'mHH_', 
                'mHHwithMET_', 
                'HighestMass_', 
                'SecondHighestMass_', 
                'FatHT_', 
                'MET_FatJet_', 
                'HighestEta_', 
                'SecondHighestEta_', 
                'DeltaEta_', 
                'DeltaPhi_']

        reference_trigger = 'HLT_AK8PFJet260'
        run_name = 'Run2'
        type = '.png'
    
        custom_order = [t + reference_trigger + '_' + run_name + type for t in files]

        dirs = f"report/{datetime.now().strftime('%d-%m-%Y')}/{suffix}"
        if not os.path.exists(dirs):
            os.makedirs(dirs, exist_ok=True)
            # filename.mkdirs(parents=True, exist_ok=True)

        filename = f"{dirs}/{sample}-eff-{suffix}.pdf"
        print(f"Creating plot grid for {sample} with filename: {filename}")
        create_plot_grid_pdf(
            source_folder=DUMMY_FOLDER,
            file_prefix=FILE_PREFIX,
            n_cols=3,
            filename=filename,
            main_title=main_title,
            subtitle=f"{sample} Efficiency Analysis",
            sort_order=custom_order
        )


In [ ]:
# STD Efficiency Plots
if __name__ == '__main__':
    samples = ["QCD", "ggF", "VBF"]
    version = "v5"
    
    if version == "v1":
        suffix = "briar"
        main_title = "Old Samples"
    elif version == "v2":
        suffix = "azura"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v3":
        suffix = "ashe"
        main_title = "New Samples - No Tau Signal Triggers"
    elif version == "v4":
        suffix = "cypress"
        main_title = "New Samples - Tau Signal Triggers"
    elif version == "v5":
        suffix = "testing"
        main_title = "New Samples - Tau Signal Triggers"

    # Create a folder and some sample files
    DUMMY_FOLDER = "result/29-07-2025/29-07-2025-" + suffix

    for sample in samples:
        FILE_PREFIX = f"STD_{sample}_TEfficiency"

        # Example with custom sort order and no title
        print("\n--- Running with custom sorted order and a title ---")
    
        # For STD plots
        files = ['HighestPt_', 
                'HT_', 
                'SecondHighestPt_', 
                'MET_pt_', 
                'mHH_', 
                'mHHwithMET_', 
                'HighestMass_', 
                'SecondHighestMass_', 
                'FatHT_', 
                'MET_FatJet_', 
                'HighestEta_', 
                'SecondHighestEta_', 
                'DeltaEta_', 
                'DeltaPhi_']

        # reference_trigger = 'HLT_AK8PFJet260'
        run_name = 'GB1_Run3'
        type = '.png'
    
        custom_order = [t +run_name + type for t in files]

        dirs = f"report/{datetime.now().strftime('%d-%m-%Y')}/{suffix}"
        if not os.path.exists(dirs):
            os.makedirs(dirs, exist_ok=True)
            # filename.mkdirs(parents=True, exist_ok=True)

        filename = f"{dirs}/{sample}-std-eff-{suffix}.pdf"
        print(f"Creating plot grid for {sample} with filename: {filename}")
        create_plot_grid_pdf(
            source_folder=DUMMY_FOLDER,
            file_prefix=FILE_PREFIX,
            n_cols=3,
            filename=filename,
            main_title=main_title,
            subtitle=f"{sample} STD Efficiency Analysis",
            sort_order=custom_order
        )


# Composing Thesis Plots

**Figure 17**: Trigger Efficiency as a function of $p+_{T0}$, $H_T$, and $p_{T}^{miss}$ in the QCD, ggF, VBF and data samples in the boosted HH analysis.

In [ ]:
if __name__ == '__main__':
    suffix = "thesis"
        
    # Create a folder and some sample files
    DUMMY_FOLDER = f"result/03-09-2025/03-09-2025-{suffix}"
    FILE_PREFIX = ""

    # Example with custom sort order and no title
    print("\n--- Running with custom sorted order ---")

    # For Distribution plots
    custom_order = ['QCD_TEfficiency_HighestPt',
                    'QCD_TEfficiency_HT',
                    'QCD_TEfficiency_SecondHighestPt',
                    'ggF_TEfficiency_HighestPt',
                    'ggF_TEfficiency_HT',
                    'ggF_TEfficiency_SecondHighestPt',
                    'VBF_TEfficiency_HighestPt',
                    'VBF_TEfficiency_HT',
                    'VBF_TEfficiency_SecondHighestPt',
                    'DATA_TEfficiency_HighestPt',
                    'DATA_TEfficiency_HT',
                    'DATA_TEfficiency_SecondHighestPt']
    
    # custom_order = [t + ".png" for t in custom_order]
    custom_order = [t + "_HLT_AK8PFJet260_" + "Run2.png" for t in custom_order]

    date = datetime.now().strftime('%d-%m-%Y')    
    dirs = f"report/{date}/{suffix}"
    if not os.path.exists(dirs):
        os.makedirs(dirs, exist_ok=True)

    filename = f"{dirs}/distribution-data-{suffix}.pdf"
    print(f"Creating plot grid for distribution with filename: {filename}")
    create_plot_grid_pdf(
        source_folder=DUMMY_FOLDER,
        file_prefix=FILE_PREFIX,
        n_cols=3,
        filename=filename,
        sort_order=custom_order
    )


--- Running with custom sorted order ---
Creating plot grid for distribution with filename: report/04-09-2025/thesis/distribution-data-thesis.pdf
Generating 'report/04-09-2025/thesis/distribution-data-thesis.pdf' with 12 plots in a 4x3 grid (DPI: 300).
PDF generation complete.


**Figure 19**: Trigger efficiency in the boosted HH analysis as a function of variables of interest in the QCD sample, containing the true efficiencies, the reference trigger measured efficiencies, the efficiencies predicted using the QCD trained gradient boosting (GB) model, and the corresponding aleatoric uncertainties, which are the standard deviations (STD) obtained with the boostrapping method.

In [4]:
if __name__ == '__main__':
    suffix = "thesis"
        
    # Create a folder and some sample files
    DUMMY_FOLDER = f"result/03-09-2025/03-09-2025-{suffix}"
    FILE_PREFIX = "STD_QCD_TEfficiency"

    # Example with custom sort order and no title
    print("\n--- Running with custom sorted order ---")

    # For Distribution plots
    custom_order = ['HighestPt_', 
                    'HT_', 
                    'SecondHighestPt_', 
                    'MET_pt_', 
                    'mHH_', 
                    'mHHwithMET_', 
                    'HighestMass_', 
                    'SecondHighestMass_', 
                    'FatHT_', 
                    'MET_FatJet_', 
                    'HighestEta_', 
                    'SecondHighestEta_', 
                    'DeltaEta_', 
                    'DeltaPhi_']
        
    # custom_order = [t + ".png" for t in custom_order]
    custom_order = [t + "GB1_Run2.png" for t in custom_order]

    date = datetime.now().strftime('%d-%m-%Y')    
    dirs = f"report/{date}/{suffix}"
    if not os.path.exists(dirs):
        os.makedirs(dirs, exist_ok=True)

    filename = f"{dirs}/applying-model-QCD-{suffix}.pdf"
    print(f"Creating plot grid for distribution with filename: {filename}")
    create_plot_grid_pdf(
        source_folder=DUMMY_FOLDER,
        file_prefix=FILE_PREFIX,
        n_cols=3,
        filename=filename,
        sort_order=custom_order
    )


--- Running with custom sorted order ---
Creating plot grid for distribution with filename: report/04-09-2025/thesis/applying-model-QCD-thesis.pdf
Generating 'report/04-09-2025/thesis/applying-model-QCD-thesis.pdf' with 14 plots in a 5x3 grid (DPI: 300).
PDF generation complete.


**Figure 20**: Trigger efficiency in the boosted HH analysis as a function of variables of interest in the ggF sample, containing the true efficiencies, the reference trigger measured efficiencies, the efficiencies predicted using the QCD trained gradient boosting (GB) model, and the corresponding aleatoric uncertainties, which are the standard deviations (STD) obtained with the boostrapping method.

In [5]:
if __name__ == '__main__':
    suffix = "thesis"
        
    # Create a folder and some sample files
    DUMMY_FOLDER = f"result/03-09-2025/03-09-2025-{suffix}"
    FILE_PREFIX = "STD_ggF_TEfficiency"

    # Example with custom sort order and no title
    print("\n--- Running with custom sorted order ---")

    # For Distribution plots
    custom_order = ['HighestPt_', 
                    'HT_', 
                    'SecondHighestPt_', 
                    'MET_pt_', 
                    'mHH_', 
                    'mHHwithMET_', 
                    'HighestMass_', 
                    'SecondHighestMass_', 
                    'FatHT_', 
                    'MET_FatJet_', 
                    'HighestEta_', 
                    'SecondHighestEta_', 
                    'DeltaEta_', 
                    'DeltaPhi_']
        
    # custom_order = [t + ".png" for t in custom_order]
    custom_order = [t + "GB1_Run2.png" for t in custom_order]

    date = datetime.now().strftime('%d-%m-%Y')    
    dirs = f"report/{date}/{suffix}"
    if not os.path.exists(dirs):
        os.makedirs(dirs, exist_ok=True)

    filename = f"{dirs}/applying-model-ggF-{suffix}.pdf"
    print(f"Creating plot grid for distribution with filename: {filename}")
    create_plot_grid_pdf(
        source_folder=DUMMY_FOLDER,
        file_prefix=FILE_PREFIX,
        n_cols=3,
        filename=filename,
        sort_order=custom_order
    )


--- Running with custom sorted order ---
Creating plot grid for distribution with filename: report/04-09-2025/thesis/applying-model-ggF-thesis.pdf
Generating 'report/04-09-2025/thesis/applying-model-ggF-thesis.pdf' with 14 plots in a 5x3 grid (DPI: 300).
PDF generation complete.


**Figure 21**: Trigger efficiency in the boosted HH analysis as a function of variables of interest in the VBF sample, containing the true efficiencies, the reference trigger measured efficiencies, the efficiencies predicted using the QCD trained gradient boosting (GB) model, and the corresponding aleatoric uncertainties, which are the standard deviations (STD) obtained with the boostrapping method.

In [6]:
if __name__ == '__main__':
    suffix = "thesis"
        
    # Create a folder and some sample files
    DUMMY_FOLDER = f"result/03-09-2025/03-09-2025-{suffix}"
    FILE_PREFIX = "STD_VBF_TEfficiency"

    # Example with custom sort order and no title
    print("\n--- Running with custom sorted order ---")

    # For Distribution plots
    custom_order = ['HighestPt_', 
                    'HT_', 
                    'SecondHighestPt_', 
                    'MET_pt_', 
                    'mHH_', 
                    'mHHwithMET_', 
                    'HighestMass_', 
                    'SecondHighestMass_', 
                    'FatHT_', 
                    'MET_FatJet_', 
                    'HighestEta_', 
                    'SecondHighestEta_', 
                    'DeltaEta_', 
                    'DeltaPhi_']
        
    # custom_order = [t + ".png" for t in custom_order]
    custom_order = [t + "GB1_Run2.png" for t in custom_order]

    date = datetime.now().strftime('%d-%m-%Y')    
    dirs = f"report/{date}/{suffix}"
    if not os.path.exists(dirs):
        os.makedirs(dirs, exist_ok=True)

    filename = f"{dirs}/applying-model-VBF-{suffix}.pdf"
    print(f"Creating plot grid for distribution with filename: {filename}")
    create_plot_grid_pdf(
        source_folder=DUMMY_FOLDER,
        file_prefix=FILE_PREFIX,
        n_cols=3,
        filename=filename,
        sort_order=custom_order
    )


--- Running with custom sorted order ---
Creating plot grid for distribution with filename: report/04-09-2025/thesis/applying-model-VBF-thesis.pdf
Generating 'report/04-09-2025/thesis/applying-model-VBF-thesis.pdf' with 14 plots in a 5x3 grid (DPI: 300).
PDF generation complete.


**Figure 22**: Trigger efficiency in the boosted HH analysis as a function of variables of interest in the data sample, containing the true efficiencies, the reference trigger measured efficiencies, the efficiencies predicted using the QCD trained gradient boosting (GB) model, and the corresponding aleatoric uncertainties, wihch are the standard deviations (STD) obtained with the boostrapping method.

In [ ]:
if __name__ == '__main__':
    suffix = "thesis"
        
    # Create a folder and some sample files
    DUMMY_FOLDER = f"result/03-09-2025/03-09-2025-{suffix}"
    FILE_PREFIX = "STD_VBF_TEfficiency"

    # Example with custom sort order and no title
    print("\n--- Running with custom sorted order ---")

    # For Distribution plots
    custom_order = ['HighestPt_', 
                    'HT_', 
                    'SecondHighestPt_', 
                    'MET_pt_', 
                    'mHH_', 
                    'mHHwithMET_', 
                    'HighestMass_', 
                    'SecondHighestMass_', 
                    'FatHT_', 
                    'MET_FatJet_', 
                    'HighestEta_', 
                    'SecondHighestEta_', 
                    'DeltaEta_', 
                    'DeltaPhi_']
        
    # custom_order = [t + ".png" for t in custom_order]
    custom_order = [t + "GB1_Run2.png" for t in custom_order]

    date = datetime.now().strftime('%d-%m-%Y')    
    dirs = f"report/{date}/{suffix}"
    if not os.path.exists(dirs):
        os.makedirs(dirs, exist_ok=True)

    filename = f"{dirs}/applying-model-VBF-{suffix}.pdf"
    print(f"Creating plot grid for distribution with filename: {filename}")
    create_plot_grid_pdf(
        source_folder=DUMMY_FOLDER,
        file_prefix=FILE_PREFIX,
        n_cols=3,
        filename=filename,
        sort_order=custom_order
    )